# <center>Interpolation Introductory Demo</center>

This demo includes a graphical interface to specify the module's properties and is meant as an introduction to it's functionality. The **Upsampling factor** and **Filter** fields, translate directly to the equivalent module properties. The input signal is a list of complex symbols. Any valid Python expression can be used for it, in the **Signal** field (defined by the *sig* variable).

The output shows the real and imaginary parts of the upsampled and filtered output signals. Note that the depending on the causality of the chosen filter, there could be a delay on the filtered signal. For example, for a typical (symmetrical around 0) Raised Cosine filter there is a $\frac{N-1}{2}$ samples of delay, where $N$ is the number of coefficients of the filter. For the same reason, only the first $S-\frac{N-1}{2m}$ symbols will be processed, where $S$ is the number of symbols in the input signal and $m$ the decimation/interpolation factor. Another detail to keep in mind is that the first $\frac{N-1}{m}$ output symbols will have lower power due to the zero initial conditions of the filter.<br>
Finally, the output also shows the magnitude and phase responses of the filter.

The **Init** button should be pressed, whenever new parameters (or a new unrelated input signal) are specified. To clear the ouput, use the Jupyter Notebook's own shortcuts. Refer to the documentation in the Help menu.

In [ ]:
%matplotlib inline

import ipywidgets as widgets
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np

import sksdr
import utils

interpolator = None
output_samples = None

def init(b):
    global interpolator, disp
    factor = factor_widget.value
    _locals = {}
    exec(filter_widget.value, None, _locals)
    coeffs = _locals['coeffs']
    interpolator = sksdr.FirInterpolator(factor, coeffs)
    with disp:
        print('Initiated with ' + repr(interpolator))

def execute(b):
    global interpolator, output_samples, disp
    _locals = {}
    exec(signal_widget.value, None, _locals)
    sig = _locals['sig']
    upsampled, out = interpolator(sig)
    fig = plt.figure(figsize=(15,10))
    gs = gridspec.GridSpec(2, 2, figure=fig)
    with disp:
        sksdr.time_plot([upsampled.real, out.real], ['Upsampled (Re)', 'Output (Re)'], [1, 1], 'Upsampling & Filter Output (Re)', fig=fig, gs=gs[0, 0])
        sksdr.time_plot([upsampled.imag, out.imag], ['Upsampled (Im)', 'Output (Im)'], [1, 1], 'Upsampling & Filter Output (Im)', fig=fig, gs=gs[0, 1])
        sksdr.freqz_plot(interpolator.coeffs, 1, 1, 'mag_db', 'Filter Frequency Response (Mag)', '|H|', fig=fig, gs=gs[1, 0])
        sksdr.freqz_plot(interpolator.coeffs, 1, 1, 'angle', 'Filter Frequency Response (Phase)', r'$\angle{H}$', fig=fig, gs=gs[1, 1])
        plt.show()
        output_samples = out
        print('Output samples available in variable "output_samples"')

style = dict(utils.description_width_style)
settings_grid = widgets.GridspecLayout(2, 3)
settings_grid[0, 0] = factor_widget = widgets.BoundedIntText(description='Upsampling factor: ', value=4, min=1, max=np.iinfo(int).max, continuous_update=False, style=style)
settings_grid[0, 1:] = filter_widget = widgets.Textarea(description='Filter (coeffs=):', value='coeffs = sksdr.rrc(sps=4, rolloff=0.5, span=10)', continuous_update=False, style=style, layout=widgets.Layout(height='auto', width='auto'))
settings_grid[1, :] = signal_widget = widgets.Textarea(description='Signal (sig=):', value='sig = np.tile(np.exp(1j * (2 * np.pi / 4 * np.arange(0,4) + np.pi / 4)), 4)', continuous_update=False, style=style, layout=widgets.Layout(height='auto', width='auto'))
init_button  = widgets.Button(description='Init', tooltip='Init')
init_button.on_click(init)
execute_button = widgets.Button(description='Execute', tooltip='Execute')
execute_button.on_click(execute)
disp = widgets.Output()
ui = widgets.VBox([
    settings_grid,
    widgets.HBox([init_button, execute_button]),
    disp
])
display(ui)